In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
from datetime import datetime


In [ ]:
# Load the pre-trained models
age_model = load_model("/content/age_detection_model.keras")
emotion_model = load_model("/content/emotion_detection_model.keras")

# Emotion labels
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]


In [ ]:
def save_data(age, emotion, allowed):
    """
    Save the detected data into a CSV file.
    """
    data = {
        "Timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
        "Age": [age],
        "Emotion": [emotion],
        "Allowed": [allowed]
    }
    df = pd.DataFrame(data)
    with open("age_emotion_data.csv", mode='a', index=False) as f:
        df.to_csv(f, header=f.tell() == 0)  # Add header only if file is new


In [ ]:
def decode_age(predicted_age_class):
    """
    Decode the predicted age class into an approximate age range.
    """
    return predicted_age_class * 15  # Age intervals of 15 (e.g., 0-14, 15-29, ...)


In [ ]:
def detect_and_display(frame):
    """
    Detect faces and predict age/emotion for each detected face.
    """
    # Load Haarcascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Preprocess face for the models
        face = frame[y:y+h, x:x+w]
        face_input = cv2.resize(face, (198, 198)) / 255.0
        face_input = np.expand_dims(face_input, axis=0)

        # Predict age
        age_probs = age_model.predict(face_input)
        predicted_age_class = age_probs.argmax()
        predicted_age = decode_age(predicted_age_class)

        # Check "Not allowed" criteria
        if predicted_age < 13 or predicted_age > 60:
            # Mark with red rectangle and display "Not allowed"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, "Not allowed", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            save_data(predicted_age, None, "No")
        else:
            # Predict emotion
            emotion_probs = emotion_model.predict(face_input)
            predicted_emotion = emotion_labels[emotion_probs.argmax()]

            # Mark with green rectangle and display age/emotion
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"Age: {predicted_age}, Emotion: {predicted_emotion}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            save_data(predicted_age, predicted_emotion, "Yes")

    return frame


In [ ]:
def main():
    """
    Run the real-time age and emotion detection.
    """
    # Open webcam
    cap = cv2.VideoCapture(1)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect and display
        processed_frame = detect_and_display(frame)
        cv2.imshow("Age and Emotion Detection", processed_frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
# Create an empty CSV file with headers
df = pd.DataFrame(columns=["Timestamp", "Age", "Emotion", "Allowed"])
df.to_csv("age_emotion_data.csv", index=False)


In [ ]:
video_capture = cv2.VideoCapture(1)


In [ ]:
# Process the captured image
try:
    filename = take_photo()
    print(f"Saved to {filename}")

    # Read the image
    img = cv2.imread(filename)
    img_resized = cv2.resize(img, (198, 198))  # Resize to the model's expected input size
    img_preprocessed = img_resized / 255.0
    img_input = np.expand_dims(img_preprocessed, axis=0)

    # Predict age
    age_pred = age_model.predict(img_input)
    print(f"Age Prediction Output: {age_pred}")  # Debugging output
    age_category = np.argmax(age_pred)
    print(f"Age Category: {age_category}")  # Debugging output

    age_ranges = ['0-14', '15-29','30-44','45-59','60-74','75-89','90-104','105-119']  # Update with your actual intervals
    if age_category < len(age_ranges):  # Check if the predicted category is valid
        predicted_age = age_ranges[age_category]
    else:
        predicted_age = "Unknown"

    # Predict emotion if within allowed age range
    if age_category == 0 or age_category > 3:  # Adjust based on your intervals
        message = "Not allowed"
        emotion = None
    else:
        emotion_pred = emotion_model.predict(img_input)
        print(f"Emotion Prediction Output: {emotion_pred}")  # Debugging output
        emotion_labels = ['Happy', 'Sad', 'Neutral']  # Replace with your emotion labels
        if len(emotion_pred) > 0:  # Ensure there is an output to handle
            emotion = emotion_labels[np.argmax(emotion_pred)]
        else:
            emotion = "Unknown"
        message = f"Emotion: {emotion}"

    # Display results
    print(f"Predicted Age: {predicted_age}")
    print(f"Message: {message}")
    if emotion:
        print(f"Emotion: {emotion}")

    # Save to CSV
    entry_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = {'Age': [predicted_age], 'Emotion': [emotion], 'Entry Time': [entry_time]}
    df = pd.DataFrame(data)
    df.to_csv('theatre_log.csv', mode='a', header=False, index=False)
    print("Data saved to theatre_log.csv")

except Exception as e:
    print(f"Error: {e}")


<IPython.core.display.Javascript object>

Saved to photo.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Age Prediction Output: [[1.1037527e-01 3.3849061e-02 6.1395075e-02 2.6629084e-01 3.7139511e-01
  1.2570420e-01 2.7346157e-02 3.4003051e-03 2.4392168e-04]]
Age Category: 4
Predicted Age: 60-74
Message: Not allowed
Data saved to theatre_log.csv
